# Data Gathering and Pre-processing

Import the required libraries...

In [70]:
import csv
import pickle
import os
import pandas as pd
import numpy as np
import nfl_data_py as nfl

## Gathering

NFL play by play data found [here](https://github.com/nflverse/nflverse-data/releases/tag/pbp). In this cell we will convert the csv files to pkl files, so they're easier to work with. This part may take a while, as the csv files are pretty big.

In [71]:
all_seasons = []
all_drives = []
starting_row = None
# loop through the play by play csv files
for i in range(1999, 2023):
    # open the csv file for the year (i is the year)
    with open(f'./pbp_csv/play_by_play_{i}.csv', 'r') as f:
        reader = csv.reader(f)

        all_games = []
        game = []
        drive = []
        drive_counter = 1
        # starting_row first row of the csv
        starting_row = next(reader)
        # loop through the rows of the csv
        for row in reader:
            # skip the first row
            if row[0] == 'play_id':
                continue
            # skip
            elif row[27] == 'GAME':
                continue
            # add the row to the game's dataframe, until the 'desc' column contains END GAME
            # desc is in column AB, which is index 27
            elif row[27] == 'END GAME':
                # add game to the list of dataframes and clear the game dataframe
                game = pd.DataFrame(game)
                game.columns = starting_row
                all_games.append(game)
                all_drives.append(drive)
                game = []
                drive_counter = 1
            else:
                if row[18] and drive_counter == int(row[18]):
                    drive.append(row)
                else:
                    all_drives.append(drive)
                    drive_counter += 1
                    drive = []
                    drive.append(row)
                    
                # add the row to the game list
                game.append(row)
        
        all_seasons.append(all_games)

In [72]:
columns_to_be_dropped = [
    'old_game_id', 
    'desc', 
    'play_id',
    'game_date',
    'ydsnet',
    'no_score_prob',
    'opp_fg_prob',
    'opp_safety_prob',
    'opp_td_prob',
    'fg_prob',
    'safety_prob',
    'td_prob',
    'extra_point_prob',
    'two_point_conversion_prob',
    'ep',
    'epa',
    'total_home_epa',
    'total_away_epa',
    'total_home_rush_epa',
    'total_away_rush_epa',
    'total_home_pass_epa',
    'total_away_pass_epa',
    'air_epa',
    'yac_epa',
    'comp_air_epa',
    'comp_yac_epa',
    'total_home_comp_air_epa',
    'total_away_comp_air_epa',
    'total_home_comp_yac_epa',
    'total_away_comp_yac_epa',
    'total_home_raw_air_epa',
    'total_away_raw_air_epa',
    'total_home_raw_yac_epa',
    'total_away_raw_yac_epa',
    'wp',
    'def_wp',
    'home_wp',
    'away_wp',
    'wpa',
    'vegas_wpa',
    'vegas_home_wpa',
    'home_wp_post',
    'away_wp_post',
    'vegas_wp',
    'vegas_home_wp',
    'total_home_rush_wpa',
    'total_away_rush_wpa',
    'total_home_pass_wpa',
    'total_away_pass_wpa',
    'air_wpa',
    'yac_wpa',
    'comp_air_wpa',
    'comp_yac_wpa',
    'total_home_comp_air_wpa',
    'total_away_comp_air_wpa',
    'total_home_comp_yac_wpa',
    'total_away_comp_yac_wpa',
    'total_home_raw_air_wpa',
    'total_away_raw_air_wpa',
    'total_home_raw_yac_wpa',
    'total_away_raw_yac_wpa',
    'passer_player_id',
    'passer_player_name',
    'receiver_player_id',
    'receiver_player_name',
    'rusher_player_id',
    'rusher_player_name',
    'lateral_receiver_player_id',
    'lateral_receiver_player_name',
    'lateral_rusher_player_id',
    'lateral_rusher_player_name',
    'lateral_sack_player_id',
    'lateral_sack_player_name',
    'interception_player_id',
    'interception_player_name',
    'lateral_interception_player_id',
    'lateral_interception_player_name',
    'punt_returner_player_id',
    'punt_returner_player_name',
    'lateral_punt_returner_player_id',
    'lateral_punt_returner_player_name',
    'kickoff_returner_player_name',
    'kickoff_returner_player_id',
    'lateral_kickoff_returner_player_id',
    'lateral_kickoff_returner_player_name',
    'punter_player_id',
    'punter_player_name',
    'kicker_player_name',
    'kicker_player_id',
    'own_kickoff_recovery_player_id',
    'own_kickoff_recovery_player_name',
    'blocked_player_id',
    'blocked_player_name',
    'tackle_for_loss_1_player_id',
    'tackle_for_loss_1_player_name',
    'tackle_for_loss_2_player_id',
    'tackle_for_loss_2_player_name',
    'qb_hit_1_player_id',
    'qb_hit_1_player_name',
    'qb_hit_2_player_id',
    'qb_hit_2_player_name',
    'forced_fumble_player_1_player_id',
    'forced_fumble_player_1_player_name',
    'forced_fumble_player_2_player_id',
    'forced_fumble_player_2_player_name',
    'solo_tackle_1_player_id',
    'solo_tackle_1_player_name',
    'solo_tackle_2_player_id',
    'solo_tackle_2_player_name',
    'assist_tackle_1_player_id',
    'assist_tackle_1_player_name',
    'assist_tackle_2_player_id',
    'assist_tackle_2_player_name',
    'assist_tackle_3_player_id',
    'assist_tackle_3_player_name',
    'assist_tackle_4_player_id',
    'assist_tackle_4_player_name',
    'tackle_with_assist_1_player_id',
    'tackle_with_assist_1_player_name',
    'tackle_with_assist_2_player_id',
    'tackle_with_assist_2_player_name',
    'pass_defense_1_player_id',
    'pass_defense_1_player_name',
    'pass_defense_2_player_id',
    'pass_defense_2_player_name',
    'fumbled_1_player_id',
    'fumbled_1_player_name',
    'fumbled_2_player_id',
    'fumbled_2_player_name',
    'fumble_recovery_1_player_id',
    'fumble_recovery_1_player_name',
    'fumble_recovery_2_player_id',
    'fumble_recovery_2_player_name',
    'sack_player_id',
    'sack_player_name',
    'half_sack_1_player_id',
    'half_sack_1_player_name',
    'half_sack_2_player_id',
    'half_sack_2_player_name',
    'penalty_player_id',
    'penalty_player_name',
    'drive_quarter_end',
    'fixed_drive',
    'fixed_drive_result',
    'drive_time_of_possession',
    'drive_play_count',
    'drive_first_downs',
    'drive_inside20',
    'drive_ended_with_score',
    'drive_game_clock_end',
    'drive_end_yard_line',
    'drive_play_id_started',
    'drive_play_id_ended',
    'away_score',
    'home_score',
    'result',
    'total',
    'total_line',
    'spread_line',
    'passer',
    'passer_jersey_number',
    'rusher',
    'rusher_jersey_number',
    'receiver',
    'receiver_jersey_number',
    'passer_id',
    'rusher_id',
    'receiver_id',
    'name',
    'jersey_number',
    'id',
    'fantasy_player_id',
    'fantasy_player_name',
    'fantasy',
    'fantasy_id',
    'qb_epa',
    'xyac_epa',
    'pass_oe'
]

## Make list of all target drives, and make all_drives into a list of dataframes

### This cell takes 30+ minutes to execute on my M2 mac, since it's grabbing every drive since 1999 and writing the drives, and targets to PKL files

In [ ]:
# Make list of all target drives
all_target_drives = []
all_drives_cleaned = []
for drive in all_drives:
    drive = pd.DataFrame(drive)
    drive.columns = starting_row
    print(drive['game_date'])
    target_drive = pd.DataFrame()
    target_drive['game_id'] = drive['game_id']
    target_drive['fixed_drive'] = drive['fixed_drive']
    target_drive['fixed_drive_result'] = drive['fixed_drive_result']
    target_drive['drive_time_of_possession'] = drive['drive_time_of_possession']
    target_drive['drive_play_count'] = drive['drive_play_count']
    target_drive['drive_first_downs'] = drive['drive_first_downs']
    target_drive['drive_inside20'] = drive['drive_inside20']
    target_drive['drive_ended_with_score'] = drive['drive_ended_with_score']
    target_drive['drive_game_clock_end'] = drive['drive_game_clock_end']
    target_drive['drive_end_yard_line'] = drive['drive_end_yard_line']
    all_target_drives.append(target_drive)
    drive.drop(columns=columns_to_be_dropped, inplace=True)
    all_drives_cleaned.append(drive)

print('making all_drives_cleaned pkl')
# write all_drives and all_target_drives to pickle files for later use
with open('all_drives_cleaned.pickle', 'wb') as f:
    pickle.dump(all_drives_cleaned, f)

print('making all_target_drives pkl')
with open('all_target_drives.pickle', 'wb') as f:
    pickle.dump(all_target_drives, f)

### Load all_drives_cleaned pkl file into variable

In [ ]:
# write all_drives_cleaned pkl to txt file to check
with open('all_drives_cleaned.pickle', 'rb') as f:
    all_drives_cleaned = pickle.load(f)
print('all_drives_cleaned pkl loaded')


## Make 2007 season PKL file for simple ML
2007 is my favorite season :D

In [ ]:
all_drives_2007_season = []
for drive in all_drives_cleaned:
    # if the first four characters of drive['game_id'] are 2007, append to all_drives_2007_season
    if drive['game_id'][0][0:4] == '2007':
        all_drives_2007_season.append(drive)
    elif int(drive['game_id'][0][0:4]) > 2007:
        break
print('all_drives_2007_season created')

# dump all_drives_2007_season to pickle file
with open('all_drives_2007_season.pickle', 'wb') as f:
    pickle.dump(all_drives_2007_season, f)
print('all_drives_2007_season pkl created')

all_target_drives_2007_season = []
# do the same for 2007 target drives
for drive in all_target_drives:
    if drive['game_id'][0][0:4] == '2007':
        all_target_drives_2007_season.append(drive)
    elif int(drive['game_id'][0][0:4]) > 2007:
        break
print('all_target_drives_2007_season created')

# dump all_target_drives_2007_season to pickle file
with open('all_target_drives_2007_season.pickle', 'wb') as f:
    pickle.dump(all_target_drives_2007_season, f)
print('all_target_drives_2007_season pkl created')

# Start here if you already have the all_drives_2007_season.pickle file

### Train model

In [77]:
# normalize each column that needs to be normalized
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load preprocessed data
with open('all_drives_cleaned.pickle', 'rb') as f:
    all_drives_cleaned = pickle.load(f)

with open('all_target_drives.pickle', 'rb') as f:
    all_target_drives = pickle.load(f)

# Let's assume 'fixed_drive_result' and 'drive_end_yard_line' are your target variables
# Concatenate all drives into single dataframes
df_drives = pd.concat(all_drives_cleaned)
df_target = pd.concat(all_target_drives)[['fixed_drive_result', 'drive_end_yard_line']]

# Split your data into training and test sets (80/20 split for example)
X_train, X_test, y_train, y_test = train_test_split(df_drives, df_target, test_size=0.2, random_state=42)

# Define your model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=[len(X_train.keys())]),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2)  # Assuming two target variables
])

# Compile the model
model.compile(
    loss='mean_squared_error',  # Suitable for continuous targets
    optimizer=tf.keras.optimizers.RMSprop(0.001)
)

# Split your data into training and test sets (80/20 split for example)
X_train, X_test, y_train, y_test = train_test_split(df_drives, df_target, test_size=0.2, random_state=42)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=10, 
    validation_split = 0.2
)

# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=2)

print("Testing set Loss: {:5.2f} ".format(loss))